In [ ]:
#!/usr/local/bin/python
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# This version of the code trains the attractor connections with a separate
# objective function than the objective function used to train all other weights
# in the network (on the prediction task).

from __future__ import print_function
import itertools
import tensorflow as tf
import numpy as np
import sys
import argparse
import datetime


% load_ext autoreload
% autoreload

from tensorflow_helpers import *
from data_generator import generate_examples, pick_task

from helper_functions import get_batches, load_pretrained_embeddings, \
    get_model_type_str, translate_ids_to_words, \
    save_results, print_into_log, print_some_translated_sentences, \
    get_training_progress_comment
from graph_init import GRU_attractor, TANH_attractor
from information_trackers import compute_entropy_fullvec


class EarlyStopper():
    def __init__(self, patience_max, disp_epoch, min_delta = 0.00):
        self.best = 1e10
        self.patience = 0  # our patience
        self.patience_max = patience_max
        self.display_epoch = disp_epoch
        self.min_delta = min_delta

    def update(self, current):
        if self.best > current:
            self.best = current
            self.patience = 0
        elif abs(self.best - current) > self.min_delta:
            self.patience += 1

    def patience_ran_out(self):
        if self.patience*self.display_epoch > self.patience_max:
            return True
        else:
            False
            
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference


ops = {
    'model_type': "TANH",  # OPTIONS: vanilla, LSTM_raw, LSTM_tensorflow, LSTM_attractor
    'hid': 100,
    'in': None,  # TBD
    'out': 1,
    #         'batch_size':n_examples, #since the sequences are 1-dimensional it's easier to just run them all at once
    'n_attractor_iterations': 15,
    'attractor_dynamics': "projection2",  # OPTIONS:  "" (for no attractor dynamics),
    #           "direct" (simple attractor weights applied to hidden states directly, trained with noise addition)
    #           "projection" (project the hidden state into a separate space via weights, do attraction, project back)
    #           "helper_hidden" (hidden-hidden neurons) - IMPORTANT: don't forget to add h_hid number
    'h_hid': 200,  # helper hidden for "helper hidden" "attractory_dynamics" mode
    'attractor_noise_level': 0.5,
    'attractor_noise_type': "bernoilli",  # OPTIONS: "gaussian", "dropout", "random_drop"

    'training_mode': "attractor_on_both",  # 'attractor_on_both',

    'attractor_regularization': "l2_regularization",  # OPTIONS: "l2_regularization", "l2_norm"
    'attractor_regularization_lambda': 0.0,

    'record_mutual_information': True,
    'problem_type': "video_classification",  # OPTIONS: parity, parity_length, majority, reber, kazakov, pos_brown, ner_german, sentiment_imdb, topic_classification, video_classification
    'masking': False,#"seq", "final"
    'prediction_type': 'final_class', #'seq', 'final', 'final_class'
    'seq_len': None,

    'save_best_model': True,
    'reshuffle_data_each_replication': False,  # relevant for POS datasets (since they are loaded from files)
    'test_partition': 0.3,
    'lrate': 1e-5,  # was 0.008

    # NLP related (pos_brown task)
    'bidirectional': False,
    'embedding_size': 100,
    'load_word_embeddings': False,
    'train_word_embeddings': False,
    'trainable_logic_symbols': 0, #make first *N* embeddings trainable(Pad, unknown, start symbols make it a separate matrix and trainable)
    'input_type': "embed",  # embed&prior, embed, prior
    'dropout': 0.4  # in range(0,1)
}

# !!!!!!!!!!!!!!!!!!!!!!
# SEQ_LEN = 12 # number of bits in input sequence
INPUT_NOISE_LEVEL = 0.1


TRAIN_ATTR_WEIGHTS_ON_PREDICTION = False
# True: train attractor weights on attractor net _and_ prediction
REPORT_BEST_TRAIN_PERFORMANCE = True
# True: save the train/test perf on the epoch for which train perf was best
LOSS_SWITCH_FREQ = 1
ops, SEQ_LEN, N_INPUT, N_CLASSES, N_TRAIN, N_TEST = pick_task(ops['problem_type'],
                                                              ops)  # task (parity, majority, reber, kazakov)

# Training Parameters

TRAINING_EPOCHS = 500
N_REPLICATIONS = 10
BATCH_SIZE = 16
DISPLAY_EPOCH = 1
EARLY_STOPPING_THRESH = 0.0 # 1e-3 for POS, 0.03 for Sentiment
EARLY_STOPPING_PATIENCE = 50  # in epochs
EARLY_STOPPING_MINIMUM_EPOCH = 0

# NOTEBOOK CODE

######### MAIN CODE #############################################################
#0.02, 0.05, 0.1, 0.2, 0.35, 0.5, 
for dataset_part in [0.2244165, 0.448833, 0.67324955, 0.897666]:
    for attractor_steps in [15, 0]:
#     for att_reg in [0.0]:
#         ops['attractor_regularization_lambda'] = att_reg
        # the tf seed needs to be within the context of the graph.
        # TODO: legacy dependency (to remove)
        N_HIDDEN = ops['hid']  # number of hidden units
        N_H_HIDDEN = ops['h_hid']
        TASK = ops['problem_type']
        ARCH = ops['model_type']  # hidden layer type: 'GRU' or 'tanh'
        NOISE_LEVEL = ops['attractor_noise_level']
        ATTRACTOR_TYPE = ops['attractor_dynamics']
        N_ATTRACTOR_STEPS = ops['n_attractor_iterations']
        # TODO: legacy dependency done
    
    
        tf.reset_default_graph()
        np.random.seed(11)
        tf.set_random_seed(11)
        ops['n_attractor_iterations'] = attractor_steps
        N_ATTRACTOR_STEPS = ops['n_attractor_iterations']

        #
        # PLACEHOLDERS
        #
        if 'pos' in ops['problem_type']:
            # X will be looked up in the embedding table, so the last dimension is just a number
            X = tf.placeholder("int64", [None, SEQ_LEN], name='X')
            # last dimension is left singular, tensorflow will expect it to be an id number, not 1-hot embed
            Y = tf.placeholder("int64", [None, SEQ_LEN], name='Y')
        elif ops['problem_type'] == 'sentiment_imdb':
             # X will be looked up in the embedding table, so the last dimension is just a number
            X = tf.placeholder("int64", [None, SEQ_LEN], name='X')
            Y = tf.placeholder("int64", [None, N_CLASSES], name='Y')
        elif ops['problem_type'] == 'topic_classification':
             # X will be looked up in the embedding table, so the last dimension is just a number
            X = tf.placeholder("int64", [None, SEQ_LEN], name='X')
            Y = tf.placeholder("int64", [None, 1], name='Y')
        elif ops['problem_type'] == 'ner_german':
            X = tf.placeholder("float", [None, SEQ_LEN, N_INPUT])
            Y = tf.placeholder("int64", [None, SEQ_LEN])
        else:  # single output 
            X = tf.placeholder("float", [None, SEQ_LEN, N_INPUT])
            Y = tf.placeholder("int64", [None, 1])
        attractor_tgt_net = tf.placeholder("float", [None, N_HIDDEN], name='attractor_tgt')

        # Embedding matrix initialization
        if 'pos' in ops['problem_type'] or 'sentiment' in ops['problem_type'] or ops['problem_type'] == "topic_classification":
            [_, _, _, _, _, _, maps] = generate_examples(SEQ_LEN, N_TRAIN, N_TEST,
                                                         INPUT_NOISE_LEVEL, TASK, ops)

            if ops['load_word_embeddings']:
                embeddings_loaded, _ = load_pretrained_embeddings('data/glove.6B.{}d.txt'.format(ops['embedding_size']),
                                                               maps, ops)
                if ops['trainable_logic_symbols'] > 0:
                    with tf.variable_scope("TASK_WEIGHTS"):
                        symbols_embedding = tf.get_variable("symb_embedding",
                                                initializer=tf.truncated_normal_initializer(stddev=0.05),
                                                shape=[ops['trainable_logic_symbols'], ops['embedding_size']],
                                                dtype=tf.float32,
                                                trainable=True)
                    
                word_embedding = tf.get_variable("embedding",
                                            initializer=embeddings_loaded,
                                            dtype=tf.float32,
                                            trainable=ops['train_word_embeddings'])
                if ops['trainable_logic_symbols'] > 0:
                    embedding = tf.concat([symbols_embedding, word_embedding], axis=0)
                else:
                    embedding = word_embedding
            else:  # initialize randomly
                embedding = tf.get_variable("embedding",
                                            initializer=tf.truncated_normal_initializer(stddev=0.05),
                                            shape=[ops['vocab_size'], ops['embedding_size']],
                                            dtype=tf.float32,
                                            trainable=ops['train_word_embeddings'])
            embed_lookup = tf.nn.embedding_lookup(embedding, X)

            # load priors information
            if ops['input_type'] == 'prior' or ops['input_type'] == 'embed&prior':
                id2prior = maps['id2prior']
                word2id = maps['word2id']
                priors = np.zeros([len(id2prior), len(id2prior[0])]).astype("float32")
                for id, prior in id2prior.items():
                    priors[id] = prior
                priors_op = tf.get_variable("priors",
                                            initializer=priors,
                                            dtype=tf.float32,
                                            trainable=False)
                prior_lookup = tf.nn.embedding_lookup(priors_op, X)

            if ops['input_type'] == 'embed':
                embed = embed_lookup
            elif ops['input_type'] == 'prior':
                embed = prior_lookup
            elif ops['input_type'] == 'embed&prior':
                embed = tf.concat([embed_lookup, prior_lookup], axis=2)

        # Graph + all the training variables
        if 'pos' in ops['problem_type']:
            net_inputs = {'X': embed, 'mask': Y, 'attractor_tgt_net': attractor_tgt_net}
        elif ops['problem_type'] == 'sentiment_imdb' or ops['problem_type'] == 'topic_classification':
            net_inputs = {'X': embed, 'mask': X, 'attractor_tgt_net': attractor_tgt_net}
        else:
            net_inputs = {'X': X, 'mask': X, 'attractor_tgt_net': attractor_tgt_net}

        if ops['model_type'] == "TANH":
            cell = TANH_attractor
        elif ops['model_type'] == "GRU":
            cell = GRU_attractor
        if ops['bidirectional']:
            G_attractors = {'forw': [], 'back': []}
            names = G_attractors.keys()
            # Forward:
            G_forw = cell(ops, inputs=net_inputs, direction='forward', suffix=names[0])
            attr_loss_op_forw = G_forw.attr_loss_op
            attr_train_op_forw = G_forw.attr_train_op
            h_clean_seq_flat_forw = G_forw.h_clean_seq_flat  # for computing entropy of states
            h_net_seq_flat_forw = G_forw.h_net_seq_flat  # -> attractor_tgt_net placeholder
            G_attractors['forw'] = {'attr_loss_op': attr_loss_op_forw, "attr_train_op": attr_train_op_forw,
                                    'h_clean_seq_flat': h_clean_seq_flat_forw, 'h_net_seq_flat': h_net_seq_flat_forw}
            G_forw_output = G_forw.output

            # Backward:
            G_back = cell(ops, inputs=net_inputs, direction='backward', suffix=names[1])
            attr_loss_op_back = G_back.attr_loss_op
            attr_train_op_back = G_back.attr_train_op
            h_clean_seq_flat_back = G_back.h_clean_seq_flat  # for computing entropy of states
            h_net_seq_flat_back = G_back.h_net_seq_flat  # -> attractor_tgt_net placeholder
            G_attractors['back'] = {'attr_loss_op': attr_loss_op_back, "attr_train_op": attr_train_op_back,
                                    'h_clean_seq_flat': h_clean_seq_flat_back, 'h_net_seq_flat': h_net_seq_flat_back}
            G_back_output = G_back.output

            
            
            # Merge: [seq_len, batch_size, n_hid*2]
            # Note that we reverse the backward cell's output to align with original direction
            # note in "final" only prediction, one less dimension
            if 'final' in ops['prediction_type']:
                merge_index = 1
            else:
                merge_index = 2
            output = tf.concat([G_forw_output, tf.reverse(G_back_output, axis=[0])], axis=merge_index)
    
            if ops['dropout'] > 0.0:
                # note keep_prob = 1.0 - drop_probability (not sure why they implemented it this way)
                # tensorflow implementation scales by 1/keep_prob automatically
                output = tf.nn.dropout(output, keep_prob=1.0 - ops['dropout'])
            else:
                output = output

            input_size_final_projection = 2 * ops['hid']
            Y_ =  project_into_output(Y, output, input_size_final_projection, ops['out'], ops)
            
            # LOSS, ACC, & TRAIN OPS
            pred_loss_op = task_loss(Y, Y_, ops)
            optimizer_pred = tf.train.AdamOptimizer(learning_rate=0.008)
            prediction_parameters = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "TASK_WEIGHTS")
            pred_train_op = optimizer_pred.minimize(pred_loss_op, var_list=prediction_parameters)
            accuracy = task_accuracy(Y, Y_, ops)
        else:
            G_attractors = {'forw': []}
            names = G_attractors.keys()
            # Forward:
            G_forw = cell(ops, inputs=net_inputs, direction='forward', suffix=names[0])
            attr_loss_op_forw = G_forw.attr_loss_op
            attr_train_op_forw = G_forw.attr_train_op
            h_clean_seq_flat_forw = G_forw.h_clean_seq_flat  # for computing entropy of states
            h_net_seq_flat_forw = G_forw.h_net_seq_flat  # -> attractor_tgt_net placeholder
            G_attractors['forw'] = {'attr_loss_op': attr_loss_op_forw, "attr_train_op": attr_train_op_forw,
                                    'h_clean_seq_flat': h_clean_seq_flat_forw, 'h_net_seq_flat': h_net_seq_flat_forw}
            G_forw_output = G_forw.output

            input_size_final_projection = ops['hid']
            
            if ops['dropout'] > 0.0:
                output = tf.nn.dropout(G_forw_output, keep_prob=1.0 - ops['dropout'])
            else:
                output = G_forw_output
            
            Y_ = project_into_output(Y, output, input_size_final_projection, ops['out'], ops)

            # LOSS, ACC, & TRAIN OPS
            pred_loss_op = task_loss(Y, Y_, ops)
            optimizer_pred = tf.train.AdamOptimizer(learning_rate=0.008)
            prediction_parameters = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "TASK_WEIGHTS")
            pred_train_op = optimizer_pred.minimize(pred_loss_op, var_list=prediction_parameters)
            accuracy = task_accuracy(Y, Y_, ops)

            
        mask_op = tf.cast(tf.sign(Y), dtype=tf.float32)
        # Initialize the variables (i.e. assign their default value)
        init = tf.global_variables_initializer()
        with tf.Session() as sess:
            # TODO: make a class for all "best" quantities (a lot of space)
            saved_train_acc = []
            saved_test_acc = []
            saved_epoch = []
            saved_att_loss = []
            saved_entropy_final = []
            saved_val_acc = []
            saved_val_loss = []
            saved_traini_loss = []
            saver = tf.train.Saver()

            # Start training
            for replication in range(N_REPLICATIONS):
                print("********** replication ", replication, " **********")
                early_stopper = EarlyStopper(EARLY_STOPPING_PATIENCE, DISPLAY_EPOCH)
                [X_full_train, Y_full_train, X_test, Y_test, X_val, Y_val, maps] = generate_examples(SEQ_LEN, N_TRAIN, N_TEST,
                                                                                           INPUT_NOISE_LEVEL, TASK, ops)
                # Take Only part of dataset:
                all_ids = range(len(X_full_train))
                np.random.shuffle(all_ids)
                train_part = int(dataset_part * len(X_full_train))
                ids_to_take = all_ids[0:train_part]
                ids_for_val = all_ids[train_part:int(train_part + 0.2*train_part)]
                if len(ids_to_take) > X_full_train.shape[0]:
                    ids_to_take = range(X_full_train.shape[0])
                X_train = X_full_train[ids_to_take, :]
                Y_train = Y_full_train[ids_to_take, :]
                
                if BATCH_SIZE < len(X_train):
                    ops['attractor_regularization_lambda'] = ops['attractor_regularization_lambda']/(len(X_train)*1.0/BATCH_SIZE)
                    print(ops['attractor_regularization_lambda'])
                
                X_val, Y_val = X_full_train[ids_for_val,:], Y_full_train[ids_for_val,:]
                
                N_TRAIN = len(X_train)
                print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

                # Log Path init-n:
                COMMENT = 'dataset_starvation_experiment'
                MODEL_NAME_FILE = '{}_(att_iter{}__bidir{}__drop{})_{}.txt'.format(ops['problem_type'],
                                                                                   ops['n_attractor_iterations'],
                                                                                   ops['bidirectional'],
                                                                                   ops['dropout'],
                                                                                   COMMENT)
                LOG_DIRECTORY = 'experiments/logs/{}'.format(MODEL_NAME_FILE)
                MODEL_DIRECTORY = 'experiments/logs/{}_{}'.format(datetime.date.today(), MODEL_NAME_FILE)
                print_into_log(LOG_DIRECTORY, get_model_type_str(ops, N_TRAIN, N_TEST, SEQ_LEN))
                print_into_log(MODEL_DIRECTORY, get_model_type_str(ops, N_TRAIN, N_TEST, SEQ_LEN), supress=True)

                sess.run(init)  # Run the initializer

                train_prediction_loss = True
                best_train_acc = -1000.
                best_test_acc = 0
                best_entropy = 0.0
                best_att_loss = 0
                best_train_loss = 0
                best_val_loss = 0.0
                best_val_acc = 0.0
                best_epoch = 0
                for epoch in range(1, TRAINING_EPOCHS + 2):
                    if (epoch - 1) % DISPLAY_EPOCH == 0:
                        # TRAIN set:
                        ploss, train_acc = batch_tensor_collect(sess, [pred_loss_op, accuracy],
                                                                X, Y, X_train, Y_train, BATCH_SIZE)
                        # TEST set:
                        test_acc = batch_tensor_collect(sess, [accuracy], X, Y, X_test, Y_test, BATCH_SIZE)[0]
                        
                        # Validation set & Early stopping:
                        ploss_val, val_acc = batch_tensor_collect(sess, [pred_loss_op, accuracy],
                                                                  X, Y, X_val, Y_val, BATCH_SIZE)
                        print(ploss_val)
                       
                        print(early_stopper.patience, early_stopper.best, ploss_val)
                        early_stopper.update(ploss_val)
                        if early_stopper.patience_ran_out():
                            print_into_log(LOG_DIRECTORY, "STOPPED EARLY AT {}".format(epoch))
                            break

                        # ATTRACTOR(s) LOSS
                        aloss = {}
                        entropy = {}
                        hid_vals_arr = batch_tensor_collect(sess, [A['h_net_seq_flat'] for att_name, A in
                                                                   G_attractors.items()],
                                                            X, Y, X_train, Y_train, BATCH_SIZE)
                        h_clean_val_arr = batch_tensor_collect(sess, [A['h_clean_seq_flat'] for att_name, A in
                                                                      G_attractors.items()],
                                                               X, Y, X_train, Y_train, BATCH_SIZE)
                        for i, attractor_name in enumerate(G_attractors.keys()):
                            A = G_attractors[attractor_name]
                            a_loss_val = []
                            n_splits = np.max([1, int(len(X_train) / BATCH_SIZE)])
                            for batch_hid_vals in np.array_split(hid_vals_arr[i], n_splits):
                                a_loss_val.append(
                                    sess.run(A['attr_loss_op'], feed_dict={attractor_tgt_net: batch_hid_vals}))
                            aloss[attractor_name] = "{:.4f}".format(np.mean(a_loss_val))

                            entropy[attractor_name] = "{:.4f}".format(
                                compute_entropy_fullvec(h_clean_val_arr[i], ops, n_bins=8))

                        # Print training information:
                        print_into_log(LOG_DIRECTORY, datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + get_training_progress_comment(epoch, ploss, aloss, ploss_val, val_acc, train_acc,
                                                                     test_acc, entropy))
                        # Update the logs:
                       
                        #                 if ops['record_mutual_information']:
                        # #                     h_attractor_val, h_clean_val = sess.run([h_attractor_collection, h_clean_seq_flat],
                        # #                                                                    feed_dict={X: X_train, Y: Y_train})
                        #                     # TODO: h_attractor_collection reshapeing masking.
                        #                     h_attractor_val = None
                        #                     h_clean_val = batch_tensor_collect(sess, [h_clean_seq_flat],
                        #                                                                         X, Y, X_train, Y_train, BATCH_SIZE)[0]
                        #                     MIS.update(ploss, aloss, train_acc, test_acc, np.tanh(hid_vals), h_attractor_val, h_clean_val)

                        if (val_acc > best_val_acc):
                            best_train_acc = train_acc
                            best_test_acc = test_acc
                            best_att_loss = aloss
                            best_epoch = epoch
                            best_val_acc = val_acc

                            best_val_loss = ploss_val
                            best_train_loss = ploss
                            if ops['save_best_model']:
                                save_path = saver.save(sess, MODEL_DIRECTORY)
                            best_entropy = entropy
                        if (1.0 - 1e-15 < 0.0):
                            print('reached_peak')
                            break

                    if epoch > 1 and LOSS_SWITCH_FREQ > 0 \
                            and (epoch - 1) % LOSS_SWITCH_FREQ == 0:
                        train_prediction_loss = not train_prediction_loss

                    # MODEL TRAINING
                    batches = get_batches(BATCH_SIZE, X_train, Y_train)
                    for (batch_x, batch_y) in batches:
                        if (LOSS_SWITCH_FREQ == 0 or train_prediction_loss):
                            # Optimize all parameters except for attractor weights
                            _ = sess.run([pred_train_op],
                                         feed_dict={X: batch_x, Y: batch_y})
                        # Attractor:
                        if (N_ATTRACTOR_STEPS > 0):
                            batch_hid_vals = sess.run([A['h_net_seq_flat'] for att_name, A in G_attractors.items()],
                                                      feed_dict={X:batch_x,  Y:batch_y})

                            for i, attractor_name in enumerate(G_attractors.keys()):
                                A = G_attractors[attractor_name]
                                _ = sess.run(A['attr_train_op'], feed_dict={attractor_tgt_net: batch_hid_vals[i]})
                print("Optimization Finished!")

                if (REPORT_BEST_TRAIN_PERFORMANCE):
                    saved_train_acc.append(best_train_acc)
                    saved_test_acc.append(best_test_acc)
                    saved_att_loss.append(best_att_loss)
                    saved_entropy_final.append(best_entropy)
                    saved_epoch.append(best_epoch)

                    saved_val_acc.append(best_val_acc)
                    saved_val_loss.append(best_val_loss)
                    saved_traini_loss.append(best_train_loss)
                else:
                    saved_train_acc.append(train_acc)
                    saved_test_acc.append(test_acc)
                    #             saved_att_loss.append(aloss)

            save_results(ops, saved_epoch, saved_train_acc, saved_test_acc, saved_att_loss, saved_entropy_final, saved_val_acc,
                 saved_val_loss, saved_traini_loss, N_TRAIN, N_TEST, SEQ_LEN, comment=COMMENT)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
L2 reg-n
********** replication  0  **********
video_classification
0.0
(499, 40, 2048) (499, 1) (99, 40, 2048) (99, 1)
Logged Successfully: 

    model_type: 		TANH bidir(False), task: video_classification
    hid: 			100,
    h_hid: 			200
    n_attractor_iterations: 	15,
    attractor_dynamics: 	projection2
    attractor_noise_level: 	0.5
    attractor_noise_type: 	bernoilli
    attractor_regu-n: 		l2_regularization(lambda:0.0)
    word_embedding: size	(100), train(False)
    dropout: 			0.4
    TRAIN/TEST_SIZE: 	499/0, SEQ_LEN: 40
Logged Successfully: 
3.2328699
0 10000000000.0 3.2328699
Logged Successfully: 
2018-05-22 09:58:22epoch=0; Loss Pred=3.2220; Val Loss=3.2329; Val Acc=0.0179; Loss Att={'forw': '1.0866'}; Train Acc=0.033; Test Acc=0.0491; Entropy={'forw': '9.4661'}; Entropy_Test=

3.3331544
0 3.2328699 3.3331544
Logged Successfully: 
2018-05-22 09:58:30epoch=1; Loss Pred=3.1693; Val Lo

2.8486102
13 2.7194934 2.8486102
Logged Successfully: 
2018-05-22 10:01:29epoch=32; Loss Pred=2.5516; Val Loss=2.8486; Val Acc=0.0804; Loss Att={'forw': '3.7346'}; Train Acc=0.197; Test Acc=0.1980; Entropy={'forw': '5.0198'}; Entropy_Test=

2.9485803
14 2.7194934 2.9485803
Logged Successfully: 
2018-05-22 10:01:35epoch=33; Loss Pred=2.6357; Val Loss=2.9486; Val Acc=0.1548; Loss Att={'forw': '8.6387'}; Train Acc=0.200; Test Acc=0.1578; Entropy={'forw': '5.4875'}; Entropy_Test=

3.0110316
15 2.7194934 3.0110316
Logged Successfully: 
2018-05-22 10:01:40epoch=34; Loss Pred=2.6627; Val Loss=3.0110; Val Acc=0.2083; Loss Att={'forw': '4.5085'}; Train Acc=0.215; Test Acc=0.1933; Entropy={'forw': '5.5474'}; Entropy_Test=

3.3046725
16 2.7194934 3.3046725
Logged Successfully: 
2018-05-22 10:01:47epoch=35; Loss Pred=2.7478; Val Loss=3.3047; Val Acc=0.1071; Loss Att={'forw': '18.9311'}; Train Acc=0.165; Test Acc=0.1429; Entropy={'forw': '5.9592'}; Entropy_Test=

3.0689638
17 2.7194934 3.0689638
Lo

Logged Successfully: 
2018-05-22 10:04:42epoch=66; Loss Pred=2.2431; Val Loss=2.7747; Val Acc=0.1518; Loss Att={'forw': '4.7031'}; Train Acc=0.236; Test Acc=0.2100; Entropy={'forw': '4.8885'}; Entropy_Test=

2.3943079
8 2.3399646 2.3943079
Logged Successfully: 
2018-05-22 10:04:49epoch=67; Loss Pred=2.2868; Val Loss=2.3943; Val Acc=0.1964; Loss Att={'forw': '17.2595'}; Train Acc=0.273; Test Acc=0.2176; Entropy={'forw': '5.8543'}; Entropy_Test=

2.284056
9 2.3399646 2.284056
Logged Successfully: 
2018-05-22 10:04:54epoch=68; Loss Pred=2.3872; Val Loss=2.2841; Val Acc=0.2827; Loss Att={'forw': '6.2527'}; Train Acc=0.247; Test Acc=0.2232; Entropy={'forw': '5.9889'}; Entropy_Test=

2.4005127
0 2.284056 2.4005127
Logged Successfully: 
2018-05-22 10:05:00epoch=69; Loss Pred=2.0448; Val Loss=2.4005; Val Acc=0.2351; Loss Att={'forw': '13.7690'}; Train Acc=0.295; Test Acc=0.2426; Entropy={'forw': '5.8518'}; Entropy_Test=

2.4894235
1 2.284056 2.4894235
Logged Successfully: 
2018-05-22 10:05:05e

2.3601432
16 2.1164296 2.3601432
Logged Successfully: 
2018-05-22 10:08:01epoch=101; Loss Pred=1.9228; Val Loss=2.3601; Val Acc=0.2500; Loss Att={'forw': '11.2589'}; Train Acc=0.319; Test Acc=0.2717; Entropy={'forw': '5.2838'}; Entropy_Test=

2.487046
17 2.1164296 2.487046
Logged Successfully: 
2018-05-22 10:08:06epoch=102; Loss Pred=2.0565; Val Loss=2.4870; Val Acc=0.2143; Loss Att={'forw': '7.1008'}; Train Acc=0.339; Test Acc=0.2750; Entropy={'forw': '5.2413'}; Entropy_Test=

2.2798092
18 2.1164296 2.2798092
Logged Successfully: 
2018-05-22 10:08:12epoch=103; Loss Pred=2.0612; Val Loss=2.2798; Val Acc=0.3065; Loss Att={'forw': '7.9199'}; Train Acc=0.362; Test Acc=0.2580; Entropy={'forw': '5.0286'}; Entropy_Test=

2.3656616
19 2.1164296 2.3656616
Logged Successfully: 
2018-05-22 10:08:17epoch=104; Loss Pred=1.8290; Val Loss=2.3657; Val Acc=0.2708; Loss Att={'forw': '5.6139'}; Train Acc=0.408; Test Acc=0.2725; Entropy={'forw': '5.0191'}; Entropy_Test=

2.1318185
20 2.1164296 2.1318185


2.276618
6 2.0444846 2.276618
Logged Successfully: 
2018-05-22 10:11:11epoch=135; Loss Pred=1.8576; Val Loss=2.2766; Val Acc=0.3363; Loss Att={'forw': '24.8637'}; Train Acc=0.388; Test Acc=0.3051; Entropy={'forw': '6.5852'}; Entropy_Test=

2.5262802
7 2.0444846 2.5262802
Logged Successfully: 
2018-05-22 10:11:16epoch=136; Loss Pred=1.8384; Val Loss=2.5263; Val Acc=0.2321; Loss Att={'forw': '14.1179'}; Train Acc=0.378; Test Acc=0.2996; Entropy={'forw': '6.8417'}; Entropy_Test=

2.530846
8 2.0444846 2.530846
Logged Successfully: 
2018-05-22 10:11:22epoch=137; Loss Pred=1.9961; Val Loss=2.5308; Val Acc=0.2887; Loss Att={'forw': '10.4843'}; Train Acc=0.380; Test Acc=0.2723; Entropy={'forw': '7.2362'}; Entropy_Test=

2.3354514
9 2.0444846 2.3354514
Logged Successfully: 
2018-05-22 10:11:27epoch=138; Loss Pred=1.8724; Val Loss=2.3355; Val Acc=0.3244; Loss Att={'forw': '9.3214'}; Train Acc=0.374; Test Acc=0.2679; Entropy={'forw': '7.2260'}; Entropy_Test=

2.4557178
10 2.0444846 2.4557178
Logg

...

Logged Successfully: 
2018-05-22 10:44:31epoch=56; Loss Pred=2.5205; Val Loss=2.8008; Val Acc=0.0893; Loss Att={'forw': '8.0776'}; Train Acc=0.176; Test Acc=0.1712; Entropy={'forw': '4.1776'}; Entropy_Test=

2.6295476
30 2.6207623 2.6295476
Logged Successfully: 
2018-05-22 10:44:37epoch=57; Loss Pred=2.6309; Val Loss=2.6295; Val Acc=0.1905; Loss Att={'forw': '24.2249'}; Train Acc=0.164; Test Acc=0.1857; Entropy={'forw': '3.8183'}; Entropy_Test=

2.7891371
31 2.6207623 2.7891371
Logged Successfully: 
2018-05-22 10:44:42epoch=58; Loss Pred=2.5367; Val Loss=2.7891; Val Acc=0.2083; Loss Att={'forw': '5.7624'}; Train Acc=0.196; Test Acc=0.1797; Entropy={'forw': '3.9852'}; Entropy_Test=

2.7526934
32 2.6207623 2.7526934
Logged Successfully: 
2018-05-22 10:44:48epoch=59; Loss Pred=2.3754; Val Loss=2.7527; Val Acc=0.2440; Loss Att={'forw': '6.6187'}; Train Acc=0.231; Test Acc=0.1853; Entropy={'forw': '4.3026'}; Entropy_Test=

2.977359
33 2.6207623 2.977359
Logged Successfully: 
2018-05-22 10:4

2.6411662
26 2.3386672 2.6411662
Logged Successfully: 
2018-05-22 10:47:36epoch=91; Loss Pred=2.2913; Val Loss=2.6412; Val Acc=0.1696; Loss Att={'forw': '11.6305'}; Train Acc=0.242; Test Acc=0.1819; Entropy={'forw': '3.9877'}; Entropy_Test=

2.5056176
27 2.3386672 2.5056176
Logged Successfully: 
2018-05-22 10:47:40epoch=92; Loss Pred=2.2351; Val Loss=2.5056; Val Acc=0.1905; Loss Att={'forw': '5.6221'}; Train Acc=0.277; Test Acc=0.1868; Entropy={'forw': '3.9754'}; Entropy_Test=

2.878205
28 2.3386672 2.878205
Logged Successfully: 
2018-05-22 10:47:46epoch=93; Loss Pred=2.3584; Val Loss=2.8782; Val Acc=0.1815; Loss Att={'forw': '24.6073'}; Train Acc=0.191; Test Acc=0.1607; Entropy={'forw': '3.6215'}; Entropy_Test=

2.914959
29 2.3386672 2.914959
Logged Successfully: 
2018-05-22 10:47:51epoch=94; Loss Pred=2.3665; Val Loss=2.9150; Val Acc=0.1161; Loss Att={'forw': '9.7746'}; Train Acc=0.212; Test Acc=0.1688; Entropy={'forw': '3.5609'}; Entropy_Test=

2.7673
30 2.3386672 2.7673
Logged Succ

3.0162797
15 2.2618122 3.0162797
Logged Successfully: 
2018-05-22 10:50:36epoch=125; Loss Pred=2.7172; Val Loss=3.0163; Val Acc=0.0893; Loss Att={'forw': '14.6824'}; Train Acc=0.174; Test Acc=0.1406; Entropy={'forw': '4.1228'}; Entropy_Test=

3.216184
16 2.2618122 3.216184
Logged Successfully: 
2018-05-22 10:50:41epoch=126; Loss Pred=2.6746; Val Loss=3.2162; Val Acc=0.1250; Loss Att={'forw': '10.2810'}; Train Acc=0.181; Test Acc=0.1551; Entropy={'forw': '4.1468'}; Entropy_Test=

2.6030335
17 2.2618122 2.6030335
Logged Successfully: 
2018-05-22 10:50:47epoch=127; Loss Pred=2.3133; Val Loss=2.6030; Val Acc=0.2143; Loss Att={'forw': '7.9532'}; Train Acc=0.213; Test Acc=0.2067; Entropy={'forw': '3.8534'}; Entropy_Test=

2.5190985
18 2.2618122 2.5190985
Logged Successfully: 
2018-05-22 10:50:51epoch=128; Loss Pred=2.2234; Val Loss=2.5191; Val Acc=0.2143; Loss Att={'forw': '9.3359'}; Train Acc=0.257; Test Acc=0.2022; Entropy={'forw': '3.9447'}; Entropy_Test=

2.7952101
19 2.2618122 2.7952101

2.560572
16 2.1583915 2.560572
Logged Successfully: 
2018-05-22 10:53:37epoch=159; Loss Pred=2.0635; Val Loss=2.5606; Val Acc=0.2232; Loss Att={'forw': '13.8246'}; Train Acc=0.292; Test Acc=0.2335; Entropy={'forw': '4.1107'}; Entropy_Test=

2.1360111
17 2.1583915 2.1360111
Logged Successfully: 
2018-05-22 10:53:42epoch=160; Loss Pred=2.0217; Val Loss=2.1360; Val Acc=0.3065; Loss Att={'forw': '7.6772'}; Train Acc=0.273; Test Acc=0.2462; Entropy={'forw': '4.1056'}; Entropy_Test=

2.6254294
0 2.1360111 2.6254294
Logged Successfully: 
2018-05-22 10:53:48epoch=161; Loss Pred=2.2043; Val Loss=2.6254; Val Acc=0.1429; Loss Att={'forw': '12.3134'}; Train Acc=0.266; Test Acc=0.2103; Entropy={'forw': '4.4319'}; Entropy_Test=

2.5786893
1 2.1360111 2.5786893
Logged Successfully: 
2018-05-22 10:53:52epoch=162; Loss Pred=2.1452; Val Loss=2.5787; Val Acc=0.1429; Loss Att={'forw': '7.6997'}; Train Acc=0.250; Test Acc=0.2221; Entropy={'forw': '4.4344'}; Entropy_Test=

2.72464
2 2.1360111 2.72464
Logged

2.4132192
32 2.1360111 2.4132192
Logged Successfully: 
2018-05-22 10:56:41epoch=193; Loss Pred=2.3549; Val Loss=2.4132; Val Acc=0.1964; Loss Att={'forw': '14.0309'}; Train Acc=0.237; Test Acc=0.2002; Entropy={'forw': '4.1163'}; Entropy_Test=

2.7816603
33 2.1360111 2.7816603
Logged Successfully: 
2018-05-22 10:56:45epoch=194; Loss Pred=2.4857; Val Loss=2.7817; Val Acc=0.2262; Loss Att={'forw': '10.1661'}; Train Acc=0.240; Test Acc=0.1893; Entropy={'forw': '4.0899'}; Entropy_Test=

2.334757
34 2.1360111 2.334757
Logged Successfully: 
2018-05-22 10:56:52epoch=195; Loss Pred=1.9748; Val Loss=2.3348; Val Acc=0.3065; Loss Att={'forw': '15.2874'}; Train Acc=0.283; Test Acc=0.2513; Entropy={'forw': '3.8213'}; Entropy_Test=

2.3434367
35 2.1360111 2.3434367
Logged Successfully: 
2018-05-22 10:56:56epoch=196; Loss Pred=1.9113; Val Loss=2.3434; Val Acc=0.2798; Loss Att={'forw': '12.2761'}; Train Acc=0.346; Test Acc=0.2301; Entropy={'forw': '3.8331'}; Entropy_Test=

2.2026205
36 2.1360111 2.20262

In [30]:
# test_cut = int(len(Y)*0.30)
# train_cut = int(len(Y) - (test_cut))
# X_train = X_bow[0:train_cut, :]
# Y_train = Y[0:train_cut]

# X_test = X[train_cut:, :]
# Y_test = Y[train_cut:]
    
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# clf = GaussianNB()
clf = SVC()
# clf.fit(X_train[:,0,:], Y_train[:,0]) 
clf.fit(np.max(X_train, axis=1), Y_train[:,0]) 


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [31]:
# Y_pred = clf.predict(X_test[:,0,:])
Y_pred = clf.predict(np.max(X_test, axis=1))
np.sum(np.equal(Y_pred, Y_test[:,0]))*1.0/2000

0.3725

In [17]:
X_train[:,0,:].shape

(1999, 2048)

In [ ]:
0.99, 0.5, 0.25, 0.75